In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DRIVE_PATH = "/content/drive/MyDrive/YandexCup/"

Mounted at /content/drive


Загружаем скачанный классификатор токсичности:

In [ ]:
!pip install -q transformers
!pip install -q natasha
!pip install -q pymorphy2
!pip install -q -U pymorphy2-dicts-ru
!pip install -q hnswlib
!pip install -q pymystem3
!pip install -q pymorphy2
!pip install -q -U pymorphy2-dicts-ru

     |████████████████████████████████| 2.9 MB 13.1 MB/s 
     |████████████████████████████████| 3.3 MB 58.6 MB/s 
     |████████████████████████████████| 636 kB 46.1 MB/s 
     |████████████████████████████████| 895 kB 66.9 MB/s 
     |████████████████████████████████| 56 kB 6.5 MB/s 
     |████████████████████████████████| 34.4 MB 51 kB/s 
     |████████████████████████████████| 55 kB 5.1 MB/s 
     |████████████████████████████████| 49 kB 8.0 MB/s 
     |████████████████████████████████| 41 kB 161 kB/s 
     |████████████████████████████████| 8.2 MB 26.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


#Score

In [ ]:
!pip install https://github.com/kpu/kenlm/archive/master.zip
!pip install -q kenlm

     \ 540 kB 2.3 MB/s
  Created wheel for kenlm: filename=kenlm-0.0.0-cp37-cp37m-linux_x86_64.whl size=2334462 sha256=149d3e4d857afcafaeea9e8dd66db0ef16b0f994cdc031811170d70a1bd1b585
  Stored in directory: /tmp/pip-ephem-wheel-cache-f668np5k/wheels/3d/aa/02/7b4a2eab5d7a2a9391bd9680dbad6270808a147bc3b7047e4e
Successfully built kenlm


In [ ]:
import numpy as np

import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch import softmax, sigmoid
from tqdm.auto import tqdm
import argparse
from pymystem3 import Mystem
import sys
from collections import Counter
from functools import partial
import kenlm

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/YandexCup/trained_roberta")

model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/YandexCup/trained_roberta").cuda()

lm = kenlm.Model('/content/drive/MyDrive/YandexCup/lm.binary')



TOXIC_CLASS=-1
TOKENIZATION_TYPE='sentencepiece'


ALLOWED_ALPHABET=list(map(chr, range(ord('а'), ord('я') + 1)))
ALLOWED_ALPHABET.extend(map(chr, range(ord('a'), ord('z') + 1)))
ALLOWED_ALPHABET.extend(list(map(str.upper, ALLOWED_ALPHABET)))
ALLOWED_ALPHABET = set(ALLOWED_ALPHABET)


def logits_to_toxic_probas(logits):
    if logits.shape[-1] > 1:
        activation = lambda x: softmax(x, -1)
    else:
        activation = sigmoid
    return activation(logits)[:, TOXIC_CLASS].cpu().detach().numpy()


def is_word_start(token):
    if TOKENIZATION_TYPE == 'sentencepiece':
        return token.startswith('▁')
    if TOKENIZATION_TYPE == 'bert':
        return not token.startswith('##')
    raise ValueError("Unknown tokenization type")


def normalize(sentence, max_tokens_per_word=20):
    def validate_char(c):
        return c in ALLOWED_ALPHABET
    
    sentence = ''.join(map(lambda c: c if validate_char(c) else ' ', sentence.lower()))
    ids = tokenizer(sentence)['input_ids']
    tokens = tokenizer.convert_ids_to_tokens(ids)[1:-1]
    
    result = []
    num_continuation_tokens = 0
    for token in tokens:
        if not is_word_start(token):
            num_continuation_tokens += 1
            if num_continuation_tokens < max_tokens_per_word:
                result.append(token.lstrip('#▁'))
        else:
            num_continuation_tokens = 0
            result.extend([' ', token.lstrip('▁#')])
    
    return ''.join(result).strip()


def iterate_batches(data, batch_size=40):
    batch = []
    for x in data:
        batch.append(x)
        if len(batch) >= batch_size:
            yield batch
            batch = []
    if len(batch) > 0:
        yield batch


def predict_toxicity_1(sentences, batch_size=5, threshold=0.5, return_scores=False, verbose=True, device='cuda'):
    results = []
    for batch in iterate_batches(sentences, batch_size):
        normlized = [normalize(sent, max_tokens_per_word=5) for sent in batch]
        tokenized = tokenizer(normlized, return_tensors='pt', padding=True, max_length=512, truncation=True)
    
        logits = model.to(device)(**{key: val.to(device) for key, val in tokenized.items()}).logits
        preds = logits_to_toxic_probas(logits)
        if not return_scores:
            preds = preds >= threshold
        results.extend(preds)
    return results


def get_w2v_indicies_1(a):
    res = []
    if isinstance(a, str):
        a = a.split()
    for w in a:
        if w in embs_voc:
            res.append((w, embs_voc[w]))
        else:
            if w.isalpha():
                lemma = morph.parse(w)[0].normal_form
                res.append((embs_voc.get(lemma), None))
    return res


def load_embeddings(path):
    embs_file = np.load(path, allow_pickle=True)
    embs_vectors = embs_file['vectors']
    embs_voc = embs_file['voc'].item()

    embs_voc_by_id = [None for i in range(len(embs_vectors))]
    for word, idx in embs_voc.items():
        if embs_voc_by_id[idx] is None:
            embs_voc_by_id[idx] = word
    return embs_vectors, embs_voc, embs_voc_by_id


def calc_embs_1(words):
    words = ' '.join(map(normalize, words))
    inds = get_w2v_indicies_1(words)
    return [(w, i if i is None else embs_vectors[i]) for w, i in inds]


def calc_single_embedding_dist(a, b):
    a_s, a_v = a  #слово, вектор
    b_s, b_v = b  #слово, вектор
    if a_s == b_s: #если слова равны
        return 0.0
    if a_v is None or b_v is None: #если одно из векторов нет - дать пизды 
        return 1.0
    a = a_v
    b = b_v
    # inexact match is punished by 0.1
    return 0.1 + 0.9 * (1 - a.dot(b) / np.linalg.norm(a) / np.linalg.norm(b)) / 2


def greedy_match_embs(a, b, max_dist=99999, cache=None, a_ind=0, b_ind=0):
    a_len = len(a) - a_ind #сколько слов мы еще не рассмотрели в а
    b_len = len(b) - b_ind #сколько слов мы еще не рассмотрели в b
    minlen = min(a_len, b_len) #минимальная длина строки
    maxlen = max(a_len, b_len) #минимальная длина строки
    if minlen == 0: 
        return np.minimum(maxlen, max_dist) 
    if maxlen - minlen >= max_dist: 
        return max_dist 
    
    if cache is None:
        cache = {}
    
    cache_key = (a_len, b_len)
    if cache_key in cache:
        return cache[cache_key]
        
    min_dist = max_dist
    
    first_dist = calc_single_embedding_dist(a[a_ind], b[b_ind])
    if max_dist >= first_dist:
        min_dist = np.minimum(min_dist, first_dist + greedy_match_embs(
            a, b, max_dist, cache, a_ind + 1, b_ind + 1
        ))
    
    if first_dist > 0 and max_dist >= 1:
        min_dist = np.minimum(min_dist, 1 + greedy_match_embs(
            a, b, max_dist, cache, a_ind + 1, b_ind
        ))
        min_dist = np.minimum(min_dist, 1 + greedy_match_embs(
            a, b, max_dist, cache, a_ind, b_ind + 1
        ))
    
    cache[cache_key] = min_dist
    
    return min_dist



def calc_semantic_distance(a, b):
    a_embs = calc_embs_1(a)
    b_embs = calc_embs_1(b)
    
    clip_distance = 5  # this clips long computations
    return np.exp(-(greedy_match_embs(a_embs, b_embs, max_dist=clip_distance) / (0.6 * np.log(1 + len(a)))) ** 2)


def distance_score(original, fixed):
    original = original.split()
    fixed = fixed.split()
    
    return calc_semantic_distance(original, fixed)


def compute_lmdiff(original, fixed):
    original_lm_logproba = lm.score(original, bos=True, eos=True)
    fixed_lm_logproba = lm.score(fixed, bos=True, eos=True)
    
    probability_fraction = 10**((fixed_lm_logproba - original_lm_logproba) / 25)
    
    return np.clip(probability_fraction, 0.0, 1.0)


def compute_score(original_sentences, fixed_sentences, threshold=0.5, batch_size=5, show_res = False):
    fixed_toxicities = predict_toxicity_1(fixed_sentences, threshold=threshold, batch_size=batch_size, return_scores=True)
    scores = []
    lmdiffs = []
    emb_dists = []
    for original_sentence, fixed_sentence, fixed_toxicity in tqdm(zip(
        original_sentences, fixed_sentences, fixed_toxicities
    ), miniters=250, disable= True):
        original_sentence = normalize(original_sentence)
        fixed_sentence = normalize(fixed_sentence)
        
        distance = distance_score(original_sentence, fixed_sentence)
        lmdiff = compute_lmdiff(original_sentence, fixed_sentence)
        
        score = (1 - fixed_toxicity) * distance * lmdiff
        
        lmdiffs.append(lmdiff)
        emb_dists.append(distance)
        scores.append(score)
        
    if show_res:
        print('average toxicity:', np.mean(fixed_toxicities), file=sys.stderr)
        print('mean lmdiff:', np.mean(lmdiffs), file=sys.stderr)
        print('mean distance_score:', np.mean(emb_dists), file=sys.stderr)
    
    return np.mean(scores)

'''
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('original_texts', type=argparse.FileType('r'))
    parser.add_argument('fixed_texts', type=argparse.FileType('r'))
    parser.add_argument('--score', type=argparse.FileType('w'))
    parser.add_argument('--model', required=True, type=str)
    parser.add_argument('--embeddings', type=str, required=True)
    parser.add_argument('--lm', type=str, required=True)
    parser.add_argument('--device', type=str, choices=['cuda', 'cpu'], default='cpu')
    
    return parser.parse_args()
'''

def new_checker(original_texts, fixed_texts, device='cuda', show_res = False):
    original_texts = list(map(str.strip, original_texts))
    fixed_texts = list(map(str.strip, fixed_texts))
    
    assert len(original_texts) == len(fixed_texts)
    
    with torch.inference_mode(True):
        return (100 * compute_score(original_texts, fixed_texts, show_res = show_res))

#Solution:

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained(DRIVE_PATH + "trained_roberta")

model = AutoModelForSequenceClassification.from_pretrained(DRIVE_PATH + "trained_roberta").cuda()

TOXIC_CLASS=-1
TOKENIZATION_TYPE='sentencepiece'


Ниже функции для применения классификатора

In [ ]:
from torch import softmax, sigmoid
import numpy as np


import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from tqdm.auto import tqdm
import argparse
from pymystem3 import Mystem
import sys
from collections import Counter
from functools import partial



ALLOWED_ALPHABET=list(map(chr, range(ord('а'), ord('я') + 1)))
ALLOWED_ALPHABET.extend(map(chr, range(ord('a'), ord('z') + 1)))
ALLOWED_ALPHABET.extend(list(map(str.upper, ALLOWED_ALPHABET)))
ALLOWED_ALPHABET = set(ALLOWED_ALPHABET)


def logits_to_toxic_probas(logits):
    if logits.shape[-1] > 1:
        activation = lambda x: softmax(x, -1)
    else:
        activation = sigmoid
    return activation(logits)[:, TOXIC_CLASS].cpu().detach().numpy()


def is_word_start(token):
    if TOKENIZATION_TYPE == 'sentencepiece':
        return token.startswith('▁')
    if TOKENIZATION_TYPE == 'bert':
        return not token.startswith('##')
    raise ValueError("Unknown tokenization type")


def normalize(sentence, max_tokens_per_word=20):
    def validate_char(c):
        return c in ALLOWED_ALPHABET
    
    sentence = ''.join(map(lambda c: c if validate_char(c) else ' ', sentence.lower()))
    ids = tokenizer(sentence)['input_ids']
    tokens = tokenizer.convert_ids_to_tokens(ids)[1:-1]
    
    result = []
    num_continuation_tokens = 0
    for token in tokens:
        if not is_word_start(token):
            num_continuation_tokens += 1
            if num_continuation_tokens < max_tokens_per_word:
                result.append(token.lstrip('#▁'))
        else:
            num_continuation_tokens = 0
            result.extend([' ', token.lstrip('▁#')])
    
    return ''.join(result).strip()


def iterate_batches(data, batch_size=40):
    batch = []
    for x in data:
        batch.append(x)
        if len(batch) >= batch_size:
            yield batch
            batch = []
    if len(batch) > 0:
        yield batch

from tqdm.auto import tqdm
def predict_toxicity(sentences, batch_size=5, threshold=0.5, return_scores=False, verbose=True, device='cuda', off_tqdm=False):
    results = []
    tqdm_fn = tqdm if verbose else lambda x, total: x
    for batch in tqdm_fn(iterate_batches(sentences, batch_size), total=np.ceil(len(sentences) / batch_size),  disable=off_tqdm):
        normlized = [normalize(sent, max_tokens_per_word=5) for sent in batch]
        tokenized = tokenizer(normlized, return_tensors='pt', padding=True, max_length=512, truncation=True)
        
        model_input = {key: val.to(device) for key, val in tokenized.items()}
        
        logits = model.to(device)(**model_input).logits
        
        preds = logits_to_toxic_probas(logits)
        
        if not return_scores:
            preds = preds >= threshold
        results.extend(preds)
    return results


In [ ]:
texts = []
raw_texts = []
with open(DRIVE_PATH + 'public_testset.txt', 'rt') as f:
    for line in f:
        raw_texts.append(line) 
        texts.append(normalize(line))

Token indices sequence length is longer than the specified maximum sequence length for this model (534 > 512). Running this sequence through the model will result in indexing errors


Вычисляем токсичность отдельных слов

In [ ]:
line_toxicities = []
line_mean_toxicities = []

for line in tqdm(np.random.choice(raw_texts, 25)):
    pred = predict_toxicity([line], return_scores=True, off_tqdm=True)
    
    line_words = normalize(line).split()
    toxi = [predict_toxicity([word], return_scores=True, off_tqdm=True) for word in line_words]
    
    line_toxicities.append(pred[0])
    line_mean_toxicities.append(np.mean(toxi))



  0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
print(f"На ({len(line_toxicities)} комментах):")
print(np.mean(line_toxicities), f"- средняя токсичность всего комментария")
print(np.mean(line_mean_toxicities), "- cредняя токсичность каждого слова в комментарии")

На (25 комментах):
0.61000323 - средняя токсичность всего комментария
0.17952412 - cредняя токсичность каждого слова в комментарии


In [ ]:
line = raw_texts[100]
pred = predict_toxicity([line], return_scores=True, off_tqdm=True)

line_words = normalize(line).split()
toxi = [predict_toxicity([word], return_scores=True, off_tqdm=True) for word in line_words]

print(line)
print(pred)
print(line_words)
print(toxi)

Можно сказать проще - кукуха едет от старческого маразма.

[0.8303647]
['можно', 'сказать', 'проще', 'кукуха', 'едет', 'от', 'старческого', 'маразма']
[[0.07630243], [0.2619401], [0.04409236], [0.56395835], [0.15138271], [0.0645212], [0.2058756], [0.63614625]]


In [ ]:
def sort_by_toxicity(words):
    toxicities = predict_toxicity(words, return_scores = True, off_tqdm=True)
    #[токсичность, индекс, слово]
    result = [[toxicities[i], i, words[i]] for i in range(len(words))]
    result.sort()
    return result

line = line = np.random.choice(raw_texts)
pred = predict_toxicity([line], return_scores=True, off_tqdm=True)

line_words = normalize(line).split()

print(sort_by_toxicity(line_words))

[[0.037139818, 15, 'у'], [0.04471565, 4, 'правильно'], [0.04769726, 5, 'два'], [0.047831487, 14, 'почему'], [0.049277287, 20, 'именно'], [0.056970093, 12, 'он'], [0.060944892, 8, 'и'], [0.060944907, 17, 'и'], [0.064153634, 11, 'ли'], [0.06441116, 19, 'расположенность'], [0.068932876, 21, 'к'], [0.07786541, 13, 'сам'], [0.08234026, 16, 'него'], [0.09730994, 3, 'р'], [0.117461026, 1, 'не'], [0.117461026, 9, 'не'], [0.13700692, 6, 'сапога'], [0.14858796, 18, 'ксении'], [0.28175718, 0, 'а'], [0.34283602, 2, 'режис'], [0.47838122, 7, 'пара'], [0.5907442, 10, 'маньяк'], [0.5997525, 22, 'маньякам']]


Ниже читаем эмбеддинги слов и описываем функции их обработки

In [ ]:
import gensim
from pymystem3 import Mystem

stemmer = Mystem()
#lemmas = stemmer.lemmatize("Красивая мама красиво мыла раму")

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [ ]:
!pip install -q pymorphy2
!pip install -q -U pymorphy2-dicts-ru

from pymystem3 import Mystem
from pymorphy2 import MorphAnalyzer
import pymorphy2


morph = pymorphy2.MorphAnalyzer(lang='ru')

In [ ]:
embs_file = np.load(DRIVE_PATH + 'embeddings_with_lemmas.npz', allow_pickle=True)
embs_vectors = embs_file['vectors']
embs_vectors_normed = embs_vectors / np.linalg.norm(embs_vectors, axis=1, keepdims=True)
embs_voc = embs_file['voc'].item()

embs_voc_by_id = [None for i in range(len(embs_vectors))]
for word, idx in embs_voc.items():
    if embs_voc_by_id[idx] is None:
        embs_voc_by_id[idx] = word

In [ ]:
words = set(embs_voc.keys())

with torch.inference_mode():
    word_toxicities = predict_toxicity(words, batch_size=100, return_scores=True)

toxicity = dict(zip(words, word_toxicities))

  0%|          | 0/1627.0 [00:00<?, ?it/s]

In [ ]:
sorted_toxicity = {k: v for k, v in sorted(toxicity.items(), key=lambda item: item[1], reverse = True)}
for k, v in list(sorted_toxicity.items())[-100:]:
    print(k ,v)

mmo 0.009779437
jpm 0.009775943
cfd 0.009774271
http 0.009772634
print 0.009772043
cf 0.009771859
8.cf 0.009771859
account 0.009770866
sncf 0.0097690765
appstore 0.009768219
dcfta 0.009765147
title 0.009763957
call-center 0.009763764
correction 0.009762546
свеже 0.009760704
documents 0.009758518
sharepoint 0.009758454
listed 0.009756879
required 0.009756584
recommended 0.009754723
board 0.009749833
дизайновый 0.009749578
postscript 0.00974957
preview 0.009746988
pocketbook 0.009745045
wcf 0.009744373
openbox 0.009742492
закладка 0.009741862
types 0.009738403
winkhaus 0.0097380495
этoгo 0.009732593
yam 0.00973132
newsonline 0.009730524
onliner.by 0.009730259
mpls 0.009729734
allview 0.009727797
jcomments 0.009724544
schedule 0.009723984
callback 0.00972145
support 0.009715264
guestbook 0.009714621
аю 0.009714375
projector 0.009711683
cfg 0.009709422
order 0.009708761
speedo 0.009708757
н::спасибо 0.009708752
registration 0.009704544
provided 0.009703988
details 0.009703582
receive 0.009

In [ ]:
def get_w2v_indicies(a):
    res = []
    if isinstance(a, str):
        a = a.split()
    for w in a:
        if w in embs_voc:
            res.append(embs_voc[w])
        else:
            
            lemma = morph.parse(w)[0].normal_form
            res.append(embs_voc.get(lemma, None))
    return res

def calc_embs(words):
    words = ' '.join(map(normalize, words))
    inds = get_w2v_indicies(words)

    return [None if i is None else embs_vectors[i] for i in inds]

Сложим эмбеддинги нетоксичных слов в kd-дерево, чтобы можно было близко искать ближайших соседей

In [ ]:
print(len(toxicity))

162690


In [ ]:
MAX_TOXICITY  = 1

non_toxicity = {word : val for word, val in toxicity.items() if val <= MAX_TOXICITY}

nontoxic_emb_inds = [ind for word, ind in embs_voc.items() if toxicity.get(word, 1.0) <= MAX_TOXICITY]
embs_vectors_normed_nontoxic = embs_vectors_normed[nontoxic_emb_inds]

print(len(non_toxicity))

162690


In [ ]:
#Super non toxic words 

MAX_TOXICITY  = 0.01

super_non_toxic = {word : val for word, val in toxicity.items() if val <= MAX_TOXICITY}

super_nontoxic_emb_inds = [ind for word, ind in embs_voc.items() if toxicity.get(word, 1.0) <= MAX_TOXICITY]
super_embs_vectors_normed_nontoxic = embs_vectors_normed[super_nontoxic_emb_inds]

print(len(super_non_toxic))
print(super_non_toxic)
print(len(super_embs_vectors_normed_nontoxic))

255
{'number': 0.009684096, 'edit': 0.00984908, 'asbook': 0.0099136, 'required': 0.009756584, 'select': 0.0096113505, 'print': 0.009772043, 'json': 0.009958467, 'services': 0.009805724, 'mp2': 0.009839244, 'guide': 0.009811462, 'product': 0.009963871, 'mmo': 0.009779437, 'directx': 0.009995624, 'sharepoint': 0.009758454, 'cfg': 0.009709422, 'explay': 0.009895934, 'address': 0.009620617, 'commander': 0.009650882, 'projects': 0.00969858, 'тoгo': 0.009811462, 'mps': 0.009665829, 'j3': 0.009945445, 'card': 0.009972742, 'available': 0.009636373, 'payment': 0.009956751, 'supported': 0.0098092845, 'receiver': 0.00961306, 'j2': 0.009945445, 'postscript': 0.00974957, 'carplay': 0.009969403, 'well': 0.00995505, 'titles': 0.009676713, 'memory': 0.009969277, 'дизайный': 0.009849666, 'types': 0.009738403, 'keycollector': 0.009655035, 'callback': 0.00972145, 'mp4': 0.009839244, 'прoкoммeнтировaть': 0.009790429, 'promotions': 0.009941017, 'support': 0.009715264, 'j': 0.009945445, 'collection': 0.0099

Функция находит самое близкое нетоксичное слово по предпосчитанным эмбеддингам слов

In [ ]:
print(*texts[:10], sep ="\n")

он скоро сдохнет и все вернется
а т у т а на заголо вок стать и р е а к ц и я тако в а рви т е вс ю сис тему и бан ки и по ли ти ку т ре б у й и т е же род н ы е наличны е н а карман вывод и т е весь бе с и нал же а по ли ти ка та ко ва мас ки це ль од на уни что жи ть наш р о д п р о с н и с ь род м о ййй же
пожалеть алису нет никакого желания не по божески да но простить е надо чтоб ей так жилось как простым мамкам одиночкам
наука в стране дураков не нужна
валерьянки выпей а то лопнешь от злости
а тебя не смутил яд на гульфике возможно он попал на трусы лешеньки прямиком с губ машеньки
бараньи отзывы а должны быть волчьи с волками жить по волчьи выть либо всем скопом вертать социализм в россию тогда будем на себя работать
с такими как все вы страшно что когда то родилась такая гадина сколько горя в семьях расстрелянных да это было давно и все равно очень страшно что на свет приходят такие нелюди и пока мы будем хоть как то оправдывать такое они эти нелюди будут появляться как вам про

In [ ]:
embs_vectors_normed_nontoxic.shape

(162690, 300)

In [ ]:
import hnswlib
import numpy as np
import pickle


num_elements, dim = super_embs_vectors_normed_nontoxic.shape

# Generating sample data
data = super_embs_vectors_normed_nontoxic
ids = np.arange(num_elements)

# Declaring index
p = hnswlib.Index(space = 'cosine', dim = dim) # possible options are l2, cosine or ip

# Initializing index - the maximum number of elements should be known beforehand
p.init_index(max_elements = num_elements, ef_construction = 400, M = 16)

# Element insertion (can be called several times):
p.add_items(data, ids)

In [ ]:
#A better one

from functools import lru_cache

@lru_cache()
def find_closest_nontoxic(word, word_toxicity, allow_self=False):
    #if toxicity.get(word, 1.0) <= threshold:
    #    return word

    #word_lemmatized = morph.parse(word)[0].normal_form
    if word not in embs_voc:
        
            return word_toxicity, word
        #else:
           # word = word_lemmatized
    

    #threshold = min(toxicity.get(word, threshold), threshold)
    
    word_emb = calc_embs([word])
    if word_emb is None or word_emb[0] is None:
        return word_toxicity, word


    query = p.knn_query(word_emb, k = 200)[0][0]


    best_candidate = word
    best_score = word_toxicity

    

    for i in query:
        candidate = embs_voc_by_id[nontoxic_emb_inds[i]]
        similarity = distance_score(word, candidate)
        
        if (1 - toxicity.get(candidate)) + (similarity) > best_score:
            
            best_score = (1 - toxicity.get(candidate)) + (similarity)
            best_candidate = candidate 
            
    return toxicity.get(candidate), best_candidate


Заменяем токсичные слова на ближайшие по эмбеддингам не-токсичные

In [ ]:
def detox(line):
    
    words = normalize(line).split()
    sorted_words = sort_by_toxicity(words) #[токсичность, индекс, слово]
    
    
    #cur_toxicity = predict_toxicity([' '.join(words)], return_scores=True, off_tqdm=True)[0]
  
    while sorted_words:
        toxic_toxicity, toxic_idx, toxic_word = sorted_words.pop()
        if toxic_toxicity > 0.77:
            fixed_word = "Спасибо"
        else:
            fixed_toxicity, fixed_word = find_closest_nontoxic(toxic_word, toxic_toxicity, allow_self = True)
        words[toxic_idx] = fixed_word
        #if fixed_word != toxic_word:
        #    sorted_words.append([fixed_toxicity, toxic_idx, fixed_word])
        #    sorted_words.sort()
        
        #cur_toxicity = predict_toxicity([' '.join(words)], return_scores=True, off_tqdm=True)[0]
        

    
    return ' '.join(words)

In [ ]:
#Super non toxic
def find_most_similar(word):
    emb_word = calc_embs([word])
    max_similarity = 0
    result = "спасибо"
    for i in p.knn_query(emb_word, k = 5)[0][0]:
        
        other_word = embs_voc_by_id[super_nontoxic_emb_inds[i]]
        
        similarity = distance_score(word, other_word)
        if similarity > max_similarity:
            max_similarity = similarity
            result = other_word
        #print(other_word, similarity, toxicity.get(other_word))
    return result


def detox_dumb(line):
    words = normalize(line).split()
    sorted_words = sort_by_toxicity(words)
    score = new_checker([line], [' '.join(words)])
    
    while sorted_words:
        toxic_toxicity, toxic_idx, toxic_word = sorted_words.pop()
        
        #if toxic_word in embs_voc:
        #    fixed_word = find_most_similar(toxic_word)
        #else:
        fixed_word = "спасибо"
        
        

        words[toxic_idx] = fixed_word
        new_score = new_checker([line], [' '.join(words)])

        if new_score >= score:
            score = new_score
        else:
            words[toxic_idx] = toxic_word
            #break
    
    return ' '.join(words)

In [ ]:
np.random.choice(list(super_non_toxic.keys()))

'cfg'

In [ ]:
detox_dumb(texts[2])

'пожалеть алису нет никакого желания не по божески да но спасибо е надо спасибо ей так жилось как простым мамкам одиночкам'

In [ ]:
Original = []
Detoxed = []
for i in tqdm(range(0, len(texts), 50)):
    line = texts[i]
    detoxed = detox_dumb(line)

    Original.append(line)
    Detoxed.append(detoxed)


new_checker(Original, Detoxed, show_res = True)

  0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

#Сравнение решений:

In [ ]:
print(len(texts))
print(len(kirill))
print(len(valya))

2500
2500
2500


In [ ]:
valya = []
kirill = []
count_valya = 0
count_kirill = 0 

with open(DRIVE_PATH + 'submits/bruteforce_hard_2_part.txt', 'rt') as f:
    for line in f:
        valya.append(line) 

with open(DRIVE_PATH + 'submits/full_thanks.txt', 'rt') as f:
    for line in f:
        kirill.append(line) 

super_result = []
for i in tqdm(range(2500)):
    kirill_score =  new_checker([texts[i]], [kirill[i]])
    valya_score = new_checker([texts[i]], [valya[i]])
    if valya_score > kirill_score:
        count_valya += 1
        super_result.append(valya[i])
        #print(valya_score,kirill_score, "Валя")
    else:
        count_kirill += 1
        super_result.append(kirill[i])
        #print(valya_score, kirill_score, "Кирилл")

    

print(count_valya, count_kirill)
new_checker(texts, super_result, show_res = True)

  0%|          | 0/2500 [00:00<?, ?it/s]

686 1814


average toxicity: 0.31801412
mean lmdiff: 0.9771276972705748
mean distance_score: 0.755816421040236


51.008772758835654

In [54]:
ultimate_solution = []

names = os.listdir(path="/content/drive/MyDrive/YandexCup/submits")
names.remove('.ipynb_checkpoints')
solutions = {name : [] for name in names}
count = {name : 0 for name in names}

#reading submits
for name in names:
    sol = []
    with open(DRIVE_PATH + f'submits/{name}', 'rt') as f:
        for line in f:
            sol.append(line)

    solutions[name] = sol[:]

for i in tqdm(range(len(texts))):
    scores = {name : 0 for name in names}
    for name in names:
        scores[name] = new_checker([texts[i]], [solutions[name][i]])

    best_sol_name = max(scores, key=scores.get)
    count[best_sol_name] += 1
    ultimate_solution.append(solutions[best_sol_name][i])


print(count)
new_checker(texts, ultimate_solution, show_res = True)

  0%|          | 0/2500 [00:00<?, ?it/s]

{'full_thanks.txt': 1799, 'bruteforce_hard_2_part.txt': 678, 'baseline_fixed.txt': 23}


average toxicity: 0.3179067
mean lmdiff: 0.9768850084694263
mean distance_score: 0.7562078477510983


51.05801094280974

In [48]:

for j in range(len(names)):
    print(solutions[j][125])  

о и у меня есть такие друзья жаль что редко видимся и прощаемся рыдаем всем желаю таких же

о и у меня есть такие друзья жаль что редко видимся и прощаемся рыдаем всем желаю таких же

о и у меня есть такие друзья жаль что редко видимся и прощаемся рыдаем всем желаю таких же

о и у меня есть такие друзья жаль что редко видимся и прощаемся рыдаем всем желаю таких же

о и у меня есть такие друзья жаль что редко видимся и прощаемся рыдаем всем желаю таких же

о и у меня есть такие друзья жаль что редко видимся и прощаемся рыдаем всем желаю таких же

о и у меня есть такие друзья жаль что редко видимся и прощаемся рыдаем всем желаю таких же

о и у меня есть такие друзья спасибо что редко видимся и прощаемся рыдаем всем желаю таких же

о и у меня есть такие друзья жаль что редко видимся и прощаемся рыдаем всем желаю таких же



In [ ]:
o = Original[2]
d = Detoxed[2]
print(o, d)
print(distance_score(o, d))

In [ ]:
fixed_texts = list(map(detox_dumb, tqdm(texts)))

with open(DRIVE_PATH + 'submits/full_thanks.txt', 'wt') as f:
    for text in fixed_texts:
        print(text, file=f)

new_checker(texts, fixed_texts, show_res = True)

In [ ]:
with open(DRIVE_PATH + 'submits/non_toxic_very_smart.txt', 'wt') as f:
    for text in fixed_texts:
        print(text, file=f)

In [ ]:
valya = []
count_valya = 0
count_kirill = 0 

with open(DRIVE_PATH + 'submits/bruteforce_hard_2_part.txt', 'rt') as f:
    for line in f:
        valya.append(line) 


In [ ]:
new_checker(texts[:len(detoxed)], detoxed, show_res = True)

In [ ]:
original_texts = []

with open(DRIVE_PATH + 'public_testset.txt', 'rt') as f:
    for line in f:
        original_texts.append(normalize(line))

text_for_eval = []
with open(DRIVE_PATH + 'submits/sorted_fixed.txt', 'rt') as f:
    for line in f:
        text_for_eval.append(normalize(line))


new_checker(original_texts, text_for_eval)

Скор, если никак не изменять комментарии:

In [ ]:
!python3.7 score.py public_testset.short.txt public_testset.short.txt  --embeddings embeddings_with_lemmas.npz --lm lm.binary --model ./trained_roberta/ --device cuda --score -

Скор бейзлайна:

In [ ]:
!python3.7 score.py public_testset.short.txt baseline_fixed.txt  --embeddings embeddings_with_lemmas.npz --lm lm.binary --model ./trained_roberta/ --device cuda --score -

Сохраним данные для бейзлайна online-задачи

##Для Online-решения

In [ ]:
!mkdir -p online_baseline

In [ ]:
import pickle as pkl

with open('./online_baseline/data.pkl', 'wb') as f:
    pkl.dump(toxicity, f)
    pkl.dump(nontoxic_emb_inds, f)